# Floor Heights Data Ingestion Example

Shell scripts that invoke the `fh-cli` CLI application to ingest data into the data model for each area of interest.

This notebook can be run with the Python environment that has the `floorheights` package installed. You will also need to install the `ipykernel` package.


## Database setup

- Deploy the docker services and migrate postgres to the latest revision of the data model.


In [ ]:
# Deploy the docker services
!docker compose up -d

# Migrate to latest revision of data model
!docker compose run --rm app alembic upgrade head

## Database ingestions

- Run shell scripts to ingest local data, this will take a few minutes.
- Make sure to update the `DATA_ROOT` variable to point to the root of the data directory.


In [ ]:
# File path to root data directory (update this!)
DATA_ROOT="/home/lng/dev/ga-floor-height-data-model/data/ffh_data/"

In [ ]:
# Wagga Wagga ingestion

# Set up file paths for Wagga Wagga data
ADDRESS=DATA_ROOT+"national_address_points/wagga/wagga_address_points.gdb/"
BUILDINGS=DATA_ROOT+"buildings/aus/overture/2024-10-23_australia_buildings.parquet"
DEM=DATA_ROOT+"dem/wagga/NSW_2020_DEM_1m.vrt"
CADASTRE=DATA_ROOT+"cadastre/wagga/Wagga_Lot_EPSG7844.gdb"
ZONING=DATA_ROOT+"zoning/wagga/wagga_land_zoning.gpkg"
NEXIS=DATA_ROOT+"nexis/NSW_building.csv"
COUNCIL_VALIDATION=DATA_ROOT+"council_validation/wagga/Final_Wagga_floor_heights.gpkg"
FSI_VALIDATION=DATA_ROOT+"fsi_validation/wagga_validation_centroid.gpkg"

# Ingest the address points
!fh-cli ingest-address-points -i {ADDRESS}

# # Ingest the building footprints, split by cadastre, and remove small buildings
!fh-cli ingest-buildings -i {BUILDINGS} -d {DEM} --split-by-cadastre {CADASTRE} --remove-small --join-land-zoning {ZONING} --land-zoning-field "LAY_CLASS"

# Join address points to building footprints
!fh-cli join-address-buildings -c {CADASTRE}

# Ingest NEXIS method floor measures, joining the largest building on each lot to a NEXIS point
!fh-cli ingest-nexis-measures -i {NEXIS} -c {CADASTRE} --join-largest-building

# Ingest Council Validation method surveyed and step counted floor measures, joining the largest building on each lot to a Validation point
!fh-cli ingest-validation-measures -i {COUNCIL_VALIDATION} --ffh-field "floor_height_m" -c {CADASTRE} --join-largest-building  --step-size 0.28 --dataset-name "Council Validation" --dataset-desc "Surveyed and step count measures provided by Wagga City Council" --dataset-src "Wagga Wagga City Council"

# Ingest FrontierSI Validation floor measures, joining the largest building on each lot to a Validation point
!fh-cli ingest-validation-measures -i {FSI_VALIDATION} --ffh-field "floor_height_m" -c {CADASTRE} --join-largest-building --method-name "LIDAR-derived" --dataset-name "FrontierSI Validation" --dataset-src "FrontierSI"

Ingesting address points
Loading address points...
Generating UUIDs...
Copying to PostgreSQL...
Address ingestion complete
Ingesting building footprints
Loading DEM...
Creating mask...
Loading building footprints...
Splitting buildings by cadastre...
Removing buildings < 30.0 m^2...
Removed 11807 buildings...
Joining land zoning attribute...
Sampling DEM with buildings...
Generating UUIDs...
Copying to PostgreSQL...
Building ingestion complete
Joining addresses to buildings
Loading cadastre...
Copying cadastre to PostgreSQL...
Performing join for building centroid addresses...
Performing join for property centroid addresses...
Joining complete
Ingesting NEXIS measures
Loading NEXIS points...
Copying NEXIS points to PostgreSQL...
Inserting GNAF records into floor_measure table...
Inserting non-GNAF records into floor_measure table...
Copying cadastre to PostgreSQL...
Joining with largest building on parcel...
NEXIS ingestion complete
Ingesting Validation measures
Loading validation poin

In [ ]:
# Launceston ingestion

# Set up file paths for Launceston data
ADDRESS=DATA_ROOT+"national_address_points/launceston/launceston_address_points.gdb/"
BUILDINGS=DATA_ROOT+"buildings/launceston/list_building_footprints_merged.gpkg"
DEM=DATA_ROOT+"dem/launceston/1m_DEM.vrt"
CADASTRE=DATA_ROOT+"cadastre/launceston/launceston_parcels_merged.gpkg"
ZONING=DATA_ROOT+"zoning/launceston/Tasmanian_Planning_Scheme_Zones_statewide.shp"
NEXIS=DATA_ROOT+"nexis/TAS_building.csv"
COUNCIL_VALIDATION=DATA_ROOT+"council_validation/launceston/LC_Final/LC_Final.shp"
FSI_VALIDATION=DATA_ROOT+"fsi_validation/launceston_validation_centroid.gpkg"

# Ingest the address points
!fh-cli ingest-address-points -i {ADDRESS}

# Ingest the building footprints, split by cadastre, and remove small buildings
!fh-cli ingest-buildings -i {BUILDINGS} -d {DEM} --split-by-cadastre {CADASTRE} --remove-small --join-land-zoning {ZONING} --land-zoning-field "ZONE"

# Join address points to building footprints
!fh-cli join-address-buildings -c {CADASTRE}

# Ingest NEXIS method floor measures, joining the largest building on each lot to a NEXIS point
!fh-cli ingest-nexis-measures -i {NEXIS} -c {CADASTRE} --join-largest-building

# Ingest Council Validation method surveyed and step counted floor measures, joining the largest building on each lot to a Validation point
!fh-cli ingest-validation-measures -i {COUNCIL_VALIDATION} --ffh-field "LCC_FLOOR" -c {CADASTRE} --join-largest-building --step-size 0.28 --flatten-cadastre --dataset-name "Council Validation" --dataset-desc "Surveyed and step count measures provided by Launceston City Council" --dataset-src "Launceston City Council"

# Ingest FrontierSI Validation floor measures, joining the largest building on each lot to a Validation point
!fh-cli ingest-validation-measures -i {FSI_VALIDATION} --ffh-field "floor_height_m" -c {CADASTRE} --join-largest-building --method-name "LIDAR-derived" --dataset-name "FrontierSI Validation" --dataset-src "FrontierSI"

Ingesting address points
Loading address points...
Generating UUIDs...
Copying to PostgreSQL...
Address ingestion complete
Ingesting building footprints
Loading DEM...
Creating mask...
Loading building footprints...
Splitting buildings by cadastre...
Removing buildings < 30.0 m^2...
Removed 6657 buildings...
Joining land zoning attribute...
Sampling DEM with buildings...
Generating UUIDs...
Copying to PostgreSQL...
Building ingestion complete
Joining addresses to buildings
Loading cadastre...
Copying cadastre to PostgreSQL...
Performing join for building centroid addresses...
Performing join for property centroid addresses...
Joining complete
Ingesting NEXIS measures
Loading NEXIS points...
Copying NEXIS points to PostgreSQL...
Inserting GNAF records into floor_measure table...
Inserting non-GNAF records into floor_measure table...
Copying cadastre to PostgreSQL...
Joining with largest building on parcel...
NEXIS ingestion complete
Ingesting Validation measures
Loading validation point

In [4]:
# Tweed Heads ingestion

# Set up file paths for Wagga Wagga data
ADDRESS=DATA_ROOT+"national_address_points/tweed/tweed_heads_address_points.gdb/"
BUILDINGS=DATA_ROOT+"buildings/aus/overture/2024-10-23_australia_buildings.parquet"
DEM=DATA_ROOT+"dem/tweed/NSW_2013_DEM_1m.vrt"
CADASTRE=DATA_ROOT+"cadastre/tweed/Tweed_Heads_Lot_EPSG7844.gdb"
ZONING=DATA_ROOT+"zoning/tweed/tweed_heads_land_zoning.gpkg"
NEXIS=DATA_ROOT+"nexis/NSW_building.csv"
COUNCIL_VALIDATION=DATA_ROOT+"council_validation/tweed/BuildingFloorLevels_FloodStudyTweed/BuildingFloorLevels_FloodStudy_floor_heights.gpkg"
FSI_VALIDATION=DATA_ROOT+"fsi_validation/tweed_validation_centroid.gpkg"


# Ingest the address points
!fh-cli ingest-address-points -i {ADDRESS}

# Ingest the building footprints, split by cadastre, and remove small buildings
!fh-cli ingest-buildings -i {BUILDINGS} -d {DEM} --split-by-cadastre {CADASTRE} --remove-small --join-land-zoning {ZONING} --land-zoning-field "LAY_CLASS"

# Join address points to building footprints
!fh-cli join-address-buildings -c {CADASTRE}

# Ingest NEXIS floor measures, joining the largest building on each lot to a NEXIS point
!fh-cli ingest-nexis-measures -i {NEXIS} -c {CADASTRE} --join-largest-building

# Ingest Council Validation surveyed floor measures, joining the largest building on each lot to a Validation point
!fh-cli ingest-validation-measures -i {COUNCIL_VALIDATION} --ffh-field "floor_height_m" -c {CADASTRE} --join-largest-building --method-name "Surveyed" --dataset-name "Council Validation" --dataset-desc "Surveyed measures provided by Tweed Shire Council" --dataset-src "Tweed Shire Council"

# Ingest FrontierSI Validation floor measures, joining the largest building on each lot to a Validation point
!fh-cli ingest-validation-measures -i {FSI_VALIDATION} --ffh-field "floor_height_m" -c {CADASTRE} --join-largest-building --method-name "LIDAR-derived" --dataset-name "FrontierSI Validation" --dataset-src "FrontierSI"

Ingesting address points
Loading address points...
Generating UUIDs...
Copying to PostgreSQL...
Address ingestion complete
Ingesting building footprints
Loading DEM...
Creating mask...
Loading building footprints...
Splitting buildings by cadastre...
Removing buildings < 30.0 m^2...
Removed 731 buildings...
Joining land zoning attribute...
Sampling DEM with buildings...
/home/lng/dev/ga-floor-height-data-model/src/floorheights/datamodel/etl.py:242: RuntimeWarning: All-NaN slice encountered
  min_height = np.nanmin(out_img)
/home/lng/dev/ga-floor-height-data-model/src/floorheights/datamodel/etl.py:243: RuntimeWarning: All-NaN slice encountered
  max_height = np.nanmax(out_img)
/home/lng/dev/ga-floor-height-data-model/src/floorheights/datamodel/etl.py:242: RuntimeWarning: All-NaN slice encountered
  min_height = np.nanmin(out_img)
/home/lng/dev/ga-floor-height-data-model/src/floorheights/datamodel/etl.py:243: RuntimeWarning: All-NaN slice encountered
  max_height = np.nanmax(out_img)
/ho

In [ ]:
# Ingest outputs from the processing workflow
FH_PARQUET=DATA_ROOT+"processing_output/all_aoi_ffh_v5_3a2a2ee6e864_s3.parquet"
PANO_PATH=DATA_ROOT+"processing_output/pano_images/"
LIDAR_PATH=DATA_ROOT+"processing_output/lidar_images/"

!fh-cli ingest-main-method-measures -i {FH_PARQUET}
!fh-cli ingest-gap-fill-measures -i {FH_PARQUET}

!fh-cli ingest-main-method-images --pano-path {PANO_PATH} --lidar-path {LIDAR_PATH} --chunksize 200

Ingesting Main Methodology measures
Loading Floor Height parquet...
Inserting records into floor_measure table...
Main methodology ingestion complete
Ingesting Gap Fill measures
Loading Floor Height parquet...
Inserting records into floor_measure table...
Gap Fill ingestion complete
Ingesting Main Methodology images
Selecting records from floor_measure table...
Ingesting panorama images...
Ingesting lidar images...
Image ingestion complete


## Stop and clear database

- Stop the docker containers and remove the docker volume to clear the data base.
- Comment the top line of cell below to run.


In [ ]:
%%script echo skipping cell  # Comment this line to run this cell

# Stop all containers, removing the postgres_data volume
!docker compose down
!docker volume rm ga-floor-height-data-model_postgres_data